Imports

In [1]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Prep Input and Output Parameters

In [2]:
## Paths Input Here
base_dir = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/rawdata'
#----------------------------------------------------------------DO NOT TOUCH----------------------------------------------------------------
analysis = fr"<subject_id>/ses-01/single_subject_atrophy"
out_dir = os.path.join(base_dir, analysis)
print('I will save each file to: \n', out_dir)

I will save each file to: 
 /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/rawdata/<subject_id>/ses-01/single_subject_atrophy


Extract Atrophy Within Region of Interest

___

## Neuroimaging File Extraction

Here's a breakdown of the inputs:

### 0. Base Directorty (CSF)
- **base_directory**: Absolute path to the base directory containing CSF files.

### 1. Cerebrospinal Fluid (CSF)
- **glob_name_pattern**: File pattern to search for CSF files.

### 2. Grey Matter
- **glob_name_pattern**: File pattern to search for grey matter files.

### 3. White Matter
- **glob_name_pattern**: File pattern to search for white matter files.

---

**Instructions**: Please fill out the `base_directory` and `glob_name_pattern` fields for each segment. This will ensure that the extraction process can locate and identify the appropriate neuroimaging files for further analysis.

---

In [3]:
# Shared Base Directory
base_directory = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/rawdata'

# Shared Glob Path
shared_glob_path = r'*/*/thresholded_tissue_segment_z_scores'

# Just the name of the tissue in the shared_glob_path directory as it appears in the directory (grey_matter, composite, etc)
tissue_to_import = ['grey_matter', 
                'white_matter', 
                'cerebrospinal_fluid', 
                'composite']

In [4]:
def import_dataframes_by_tissue(base_directory, shared_glob_path, tissue_to_import):
    """
    Imports dataframes based on tissue types from specified directories and glob name patterns.
    
    Parameters:
    - base_directory (str): The base directory where the data resides.
    - shared_glob_path (str): The shared directory path for all tissues.
    - tissue_to_import (list): List of tissues to be imported.
    
    Returns:
    - dict: A dictionary containing dataframes for each specified tissue.
    """
    
    from calvin_utils.file_utils.import_matrices import import_matrices_from_folder

    segments_dict = {}
    for tissue in tissue_to_import:
        glob_path = os.path.join(shared_glob_path, ('*'+tissue+'*'))
        segments_dict[tissue] = glob_path

    dataframes_dict = {}
    nan_handler = {'nan': 0, 'posinf':20, 'neginf':-20}
    for k, v in segments_dict.items():
        dataframes_dict[k] = import_matrices_from_folder(connectivity_path=base_directory, file_pattern=v, convert_nan_to_num=nan_handler)
        print(f'Imported data {k} data with {dataframes_dict[k].shape[0]} voxels and {dataframes_dict[k].shape[1]} patients')
        print(f'Example filename per subject: {dataframes_dict[k].columns[0]}')
        print('\n--------------------------------\n')

    return dataframes_dict


In [5]:
imported_dataframes_by_tissue = import_dataframes_by_tissue(base_directory, shared_glob_path, tissue_to_import)

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/rawdata/*/*/thresholded_tissue_segment_z_scores/*grey_matter*
Imported data grey_matter data with 902629 voxels and 50 patients
Example filename per subject: sub-130_grey_matter_generated_nifti.nii

--------------------------------

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/rawdata/*/*/thresholded_tissue_segment_z_scores/*white_matter*
Imported data white_matter data with 902629 voxels and 50 patients
Example filename per subject: sub-130_white_matter_generated_nifti.nii

--------------------------------

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/rawdata/*/*/thresholded_tissue_segment_z_scores/*cerebrospinal_fluid*
Imported data cerebrospinal_fluid data with 902629 voxels and 50 patients
Example filename per subject: sub-130_cerebrospinal_fluid_generated_nifti.nii

-----------------

**Extract Subject ID From File Names**
Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID.
2) Proceeding the subject ID. 

In [6]:
def rename_dataframe_subjects(dataframes_dict, preceding_id, proceeding_id):
    """
    Renames the subjects in the provided dataframes based on the split commands.

    Parameters:
    - dataframes_dict (dict): A dictionary containing dataframes with subjects to be renamed.
    - preceding_id (str): The delimiter for taking the part after the split.
    - proceeding_id (str): The delimiter for taking the part before the split.

    Returns:
    - dict: A dictionary containing dataframes with subjects renamed.
    """
    
    from calvin_utils.file_utils.dataframe_utilities import extract_and_rename_subject_id

    split_command_dict = {preceding_id: 1, proceeding_id: 0}
    
    for k, v in dataframes_dict.items():
        dataframes_dict[k] = extract_and_rename_subject_id(dataframe=dataframes_dict[k], split_command_dict=split_command_dict)
        print('Dataframe: ', k)
        display(dataframes_dict[k])
        print('------------- \n')

    return dataframes_dict


In [7]:
preceding_id = 'sub-'
proceeding_id = '_'

In [8]:
thresholded_atrophy_df_dict = rename_dataframe_subjects(imported_dataframes_by_tissue, preceding_id, proceeding_id)

Dataframe:  grey_matter


,130,137,108,101,106,139,107,138,109,131,...,143,120,118,127,111,129,116,142,145,1201
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 

Dataframe:  white_matter


,130,137,108,101,106,139,107,138,109,131,...,143,120,118,127,111,129,116,142,145,1201
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 

Dataframe:  cerebrospinal_fluid


,130,137,108,101,106,139,107,138,109,131,...,143,120,118,127,111,129,116,142,145,1201
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 

Dataframe:  composite


,130,137,108,101,106,139,107,138,109,131,...,143,120,118,127,111,129,116,142,145,1201
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 



# Extract Region of Interest Atrophy

Import Region of Interest Masks

In [9]:
folder_to_import_from = '/Users/cu135/Dropbox (Partners HealthCare)/resources/atlases/yeo_thick_seeds/big_seven'
shared_glob_pattern = '*.nii'

In [10]:
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
region_of_interest_df = import_matrices_from_folder(connectivity_path=folder_to_import_from, file_pattern=shared_glob_pattern)
region_of_interest_df

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/atlases/yeo_thick_seeds/big_seven/*.nii


,07_default.nii,01_visual_seed.nii,05_limbic.nii,02_somatomotor_seed.nii,03_dorsal_attention.nii,04_ventral_attention.nii,06_frontoparietal.nii
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Extract Damage Scores Per Region of Interest

In [11]:
def calculate_damage_scores(thresholded_atrophy_df_dict, region_of_interest_df):
    """
    Calculates damage scores for each region of interest based on thresholded atrophy data.
    
    Parameters:
    - thresholded_atrophy_df_dict (dict): Dictionary containing dataframes with thresholded atrophy data.
    - region_of_interest_df (DataFrame): DataFrame containing regions of interest.
    
    Returns:
    - dict: A dictionary containing damage scores for each region of interest for each patient.
    """
    
    damage_df_dict = {}
    
    for k, v in thresholded_atrophy_df_dict.items():
        
        # Initialize the Dataframe
        damage_df_dict[k] = pd.DataFrame(index=v.columns, columns=region_of_interest_df.columns)
        
        # Iterate Over Each Region of Interest
        for matrix in damage_df_dict[k].columns:
            # Extract Damage Score for Each Patient
            for subject in damage_df_dict[k].index:
                # Mask the subject dataframe to the matrix at hand
                intersection = v[subject].where(region_of_interest_df[matrix] > 0, 0)
                
                # Weight the overlapping components by multiplication
                weighted_overlap = intersection * region_of_interest_df[matrix]
                
                # Assess overall damage score
                damage_score = weighted_overlap.sum()
                damage_df_dict[k].loc[subject, matrix] = damage_score
                
        print('Dataframe: ', k)
        display(damage_df_dict[k])
        print('------------- \n')
    
    return damage_df_dict


In [12]:
damage_df_dict = calculate_damage_scores(thresholded_atrophy_df_dict, region_of_interest_df)


Dataframe:  grey_matter


,07_default.nii,01_visual_seed.nii,05_limbic.nii,02_somatomotor_seed.nii,03_dorsal_attention.nii,04_ventral_attention.nii,06_frontoparietal.nii
130,-3454.599843,-3115.130458,-1741.86852,-2304.964156,-2467.201524,-1690.100014,-2485.750195
137,-18862.602922,-7436.758739,-3721.461204,-11155.497347,-8890.355496,-7789.708139,-12670.071527
108,-8107.05032,-4703.629885,-3389.400475,-3244.698615,-2471.534292,-1827.228059,-2678.637152
101,-6270.805613,-4621.706587,-2785.906135,-2547.638039,-2233.348331,-1605.735372,-2763.053388
106,-2171.991575,-2394.039382,-1181.315459,-540.780031,-1488.60791,-797.081924,-1820.903033
139,-6360.066675,-5683.62138,-765.854754,-6806.557983,-3821.112776,-3007.795181,-3647.386189
107,-7057.018514,-6628.774196,-5403.538442,-4475.825124,-3829.888645,-2505.786656,-3740.9724
138,-10113.605767,-4760.390901,-7211.725571,-7580.63481,-4617.448618,-4198.749244,-6553.997013
109,-6662.306683,-5023.988879,-4740.987264,-2265.663935,-2041.301181,-2479.113512,-3235.212698
131,-3700.051655,-1934.448573,-2753.894165,-1245.060733,-1110.667754,-1563.082527,-2392.763987


------------- 

Dataframe:  white_matter


,07_default.nii,01_visual_seed.nii,05_limbic.nii,02_somatomotor_seed.nii,03_dorsal_attention.nii,04_ventral_attention.nii,06_frontoparietal.nii
130,-178.234988,-252.122354,-136.92199,-69.832776,-161.062638,-92.957116,-219.942696
137,-894.278807,-467.966195,-857.47298,-64.35809,-466.387103,-83.891197,-468.219003
108,-1811.705724,-920.876664,-746.509263,-2862.964577,-410.554899,-1052.096194,-572.920419
101,-338.814533,-207.506832,-350.510831,-370.559216,-194.979006,-222.541308,-202.642969
106,-1223.273812,-831.893366,-538.410954,-358.271343,-418.733141,-380.559003,-680.509347
139,-156.282422,-169.698458,-94.164725,-207.742874,-293.145186,-141.875045,-76.277185
107,-994.79027,-682.535048,-565.295653,-413.534831,-564.389664,-380.022258,-631.275725
138,-1315.478981,-600.741455,-1223.393602,-136.477656,-319.236472,-243.274401,-410.564441
109,-582.022142,-794.321683,-376.252447,-573.953527,-464.076117,-462.652188,-209.10823
131,-563.73492,-533.331374,-208.436536,-225.219388,-147.676254,-269.327125,-306.145937


------------- 

Dataframe:  cerebrospinal_fluid


,07_default.nii,01_visual_seed.nii,05_limbic.nii,02_somatomotor_seed.nii,03_dorsal_attention.nii,04_ventral_attention.nii,06_frontoparietal.nii
130,9787.564682,12662.34177,4640.284987,3812.643049,5410.083865,4098.849306,6778.60926
137,2468.881976,1796.997549,447.546165,243.466245,2572.529643,211.752293,2227.907513
108,11916.576205,11953.440405,5247.425554,9444.495133,5772.577113,6165.476253,8286.414612
101,15310.064023,13137.737754,5308.167206,6530.137526,8517.089994,4291.447046,7349.215393
106,2690.561112,3154.049577,468.856877,836.618023,1427.032366,630.12343,2411.333388
139,7857.780837,5971.095983,2480.721721,7083.036124,7118.927216,4122.188423,5719.341504
107,4426.463716,5920.364036,2389.605977,2800.250174,3464.756045,1999.907275,2767.007182
138,2881.358019,1411.867672,2286.524934,1675.964523,1710.028769,1201.382854,1732.078893
109,17923.662841,9313.825373,6170.661867,9467.661483,8205.380507,9089.650792,13307.331159
131,12413.554404,9704.761304,6327.942012,6316.731143,6311.840848,6403.984554,7544.126888


------------- 

Dataframe:  composite


,07_default.nii,01_visual_seed.nii,05_limbic.nii,02_somatomotor_seed.nii,03_dorsal_attention.nii,04_ventral_attention.nii,06_frontoparietal.nii
130,13407.167646,15979.577064,6499.266731,6187.439981,8018.928787,5879.470459,9466.614271
137,22181.711697,9699.551644,5026.480349,11463.321682,11911.856731,8085.351629,15356.255272
108,21710.049405,17434.430746,9310.343987,15141.336205,8642.844347,8945.818585,11515.875431
101,21903.170031,17957.673191,8404.868227,9439.419785,10936.447981,6115.085079,10314.911749
106,6082.399537,6377.536797,2188.58329,1735.669396,3331.95168,1807.764356,4907.161461
139,14355.62651,11814.335256,3338.023697,14030.008335,11160.6434,7241.547229,9425.910449
107,12465.439979,13215.160937,8343.393797,7678.161769,7852.485617,4876.8917,7134.532788
138,14282.490693,6755.741962,10671.394202,9388.230279,6638.687898,5637.29579,8694.398513
109,25161.368954,15101.270296,11257.506771,12292.241319,10700.715261,12022.943504,16740.088351
131,16639.917567,12131.877789,9165.831671,7775.582144,7564.854388,8214.833815,10230.680685


------------- 



Extract Total Atrophy Volume

In [13]:
def calculate_total_atrophy_voxels(thresholded_atrophy_df_dict, damage_df_dict):
    """
    Calculates the total number of atrophy voxels within a mask and updates the damage_df_dict with this information.
    
    Parameters:
    - thresholded_atrophy_df_dict (dict): Dictionary containing dataframes with thresholded atrophy data.
    - damage_df_dict (dict): Dictionary containing dataframes to which the 'Total Atrophy Voxels' column will be added.
    
    Returns:
    - dict: The updated damage_df_dict with a new column 'Total Atrophy Voxels'.
    """
    
    import numpy as np
    from nimlab import datasets as nimds
    
    # Get the mask and brain indices
    mni_mask = nimds.get_img("mni_icbm152")
    mask_data = mni_mask.get_fdata().flatten()
    brain_indices = np.where(mask_data > 0)[0]
    
    for k, v in thresholded_atrophy_df_dict.items():
        # initialize the column
        damage_df_dict[k]['Total Atrophy Voxels'] = np.nan
        for patient_id in v.columns:
            # Filter the dataframe using brain indices
            filtered_df = v[patient_id].iloc[brain_indices]
            damage_df_dict[k].loc[patient_id, 'Total Atrophy Voxels'] = np.count_nonzero(filtered_df)
        
        print('Dataframe: ', k)
        display(damage_df_dict[k])
        print('------------- \n')
    
    return damage_df_dict


In [14]:
damage_df_dict = calculate_total_atrophy_voxels(thresholded_atrophy_df_dict, damage_df_dict)

Dataframe:  grey_matter


,07_default.nii,01_visual_seed.nii,05_limbic.nii,02_somatomotor_seed.nii,03_dorsal_attention.nii,04_ventral_attention.nii,06_frontoparietal.nii,Total Atrophy Voxels
130,-3454.599843,-3115.130458,-1741.86852,-2304.964156,-2467.201524,-1690.100014,-2485.750195,6992.0
137,-18862.602922,-7436.758739,-3721.461204,-11155.497347,-8890.355496,-7789.708139,-12670.071527,26781.0
108,-8107.05032,-4703.629885,-3389.400475,-3244.698615,-2471.534292,-1827.228059,-2678.637152,10519.0
101,-6270.805613,-4621.706587,-2785.906135,-2547.638039,-2233.348331,-1605.735372,-2763.053388,8731.0
106,-2171.991575,-2394.039382,-1181.315459,-540.780031,-1488.60791,-797.081924,-1820.903033,4426.0
139,-6360.066675,-5683.62138,-765.854754,-6806.557983,-3821.112776,-3007.795181,-3647.386189,13139.0
107,-7057.018514,-6628.774196,-5403.538442,-4475.825124,-3829.888645,-2505.786656,-3740.9724,13976.0
138,-10113.605767,-4760.390901,-7211.725571,-7580.63481,-4617.448618,-4198.749244,-6553.997013,20272.0
109,-6662.306683,-5023.988879,-4740.987264,-2265.663935,-2041.301181,-2479.113512,-3235.212698,9663.0
131,-3700.051655,-1934.448573,-2753.894165,-1245.060733,-1110.667754,-1563.082527,-2392.763987,6448.0


------------- 

Dataframe:  white_matter


,07_default.nii,01_visual_seed.nii,05_limbic.nii,02_somatomotor_seed.nii,03_dorsal_attention.nii,04_ventral_attention.nii,06_frontoparietal.nii,Total Atrophy Voxels
130,-178.234988,-252.122354,-136.92199,-69.832776,-161.062638,-92.957116,-219.942696,766.0
137,-894.278807,-467.966195,-857.47298,-64.35809,-466.387103,-83.891197,-468.219003,2147.0
108,-1811.705724,-920.876664,-746.509263,-2862.964577,-410.554899,-1052.096194,-572.920419,4757.0
101,-338.814533,-207.506832,-350.510831,-370.559216,-194.979006,-222.541308,-202.642969,1120.0
106,-1223.273812,-831.893366,-538.410954,-358.271343,-418.733141,-380.559003,-680.509347,2343.0
139,-156.282422,-169.698458,-94.164725,-207.742874,-293.145186,-141.875045,-76.277185,779.0
107,-994.79027,-682.535048,-565.295653,-413.534831,-564.389664,-380.022258,-631.275725,3050.0
138,-1315.478981,-600.741455,-1223.393602,-136.477656,-319.236472,-243.274401,-410.564441,2549.0
109,-582.022142,-794.321683,-376.252447,-573.953527,-464.076117,-462.652188,-209.10823,2654.0
131,-563.73492,-533.331374,-208.436536,-225.219388,-147.676254,-269.327125,-306.145937,1263.0


------------- 

Dataframe:  cerebrospinal_fluid


,07_default.nii,01_visual_seed.nii,05_limbic.nii,02_somatomotor_seed.nii,03_dorsal_attention.nii,04_ventral_attention.nii,06_frontoparietal.nii,Total Atrophy Voxels
130,9787.564682,12662.34177,4640.284987,3812.643049,5410.083865,4098.849306,6778.60926,12566.0
137,2468.881976,1796.997549,447.546165,243.466245,2572.529643,211.752293,2227.907513,3627.0
108,11916.576205,11953.440405,5247.425554,9444.495133,5772.577113,6165.476253,8286.414612,13124.0
101,15310.064023,13137.737754,5308.167206,6530.137526,8517.089994,4291.447046,7349.215393,15272.0
106,2690.561112,3154.049577,468.856877,836.618023,1427.032366,630.12343,2411.333388,3032.0
139,7857.780837,5971.095983,2480.721721,7083.036124,7118.927216,4122.188423,5719.341504,10233.0
107,4426.463716,5920.364036,2389.605977,2800.250174,3464.756045,1999.907275,2767.007182,6087.0
138,2881.358019,1411.867672,2286.524934,1675.964523,1710.028769,1201.382854,1732.078893,3142.0
109,17923.662841,9313.825373,6170.661867,9467.661483,8205.380507,9089.650792,13307.331159,21124.0
131,12413.554404,9704.761304,6327.942012,6316.731143,6311.840848,6403.984554,7544.126888,12997.0


------------- 

Dataframe:  composite


,07_default.nii,01_visual_seed.nii,05_limbic.nii,02_somatomotor_seed.nii,03_dorsal_attention.nii,04_ventral_attention.nii,06_frontoparietal.nii,Total Atrophy Voxels
130,13407.167646,15979.577064,6499.266731,6187.439981,8018.928787,5879.470459,9466.614271,17240.0
137,22181.711697,9699.551644,5026.480349,11463.321682,11911.856731,8085.351629,15356.255272,30920.0
108,21710.049405,17434.430746,9310.343987,15141.336205,8642.844347,8945.818585,11515.875431,23894.0
101,21903.170031,17957.673191,8404.868227,9439.419785,10936.447981,6115.085079,10314.911749,20849.0
106,6082.399537,6377.536797,2188.58329,1735.669396,3331.95168,1807.764356,4907.161461,8922.0
139,14355.62651,11814.335256,3338.023697,14030.008335,11160.6434,7241.547229,9425.910449,21691.0
107,12465.439979,13215.160937,8343.393797,7678.161769,7852.485617,4876.8917,7134.532788,20189.0
138,14282.490693,6755.741962,10671.394202,9388.230279,6638.687898,5637.29579,8694.398513,24304.0
109,25161.368954,15101.270296,11257.506771,12292.241319,10700.715261,12022.943504,16740.088351,28436.0
131,16639.917567,12131.877789,9165.831671,7775.582144,7564.854388,8214.833815,10230.680685,17929.0


------------- 



Organize the Subjects

In [15]:
def sort_dataframes_by_index(damage_df_dict):
    """
    Attempts to convert the index of each dataframe in damage_df_dict to integers 
    and then sorts the dataframe by its index in ascending order.
    
    Parameters:
    - damage_df_dict (dict): Dictionary containing dataframes to be sorted.
    
    Returns:
    - dict: The sorted damage_df_dict.
    """
    
    sorted_df_dict = {}
    
    for k, df in damage_df_dict.items():
        try:
            # Convert index to integers
            df.index = df.index.astype(int)
            
            # Sort dataframe by index
            sorted_df = df.sort_index(ascending=True)
            sorted_df_dict[k] = sorted_df
            
            # Display the results
            print('Dataframe: ', k)
            display(sorted_df)
            print('------------- \n')
            
        except ValueError:
            # If conversion to integer fails, just use the original dataframe
            sorted_df_dict[k] = df

    return sorted_df_dict


In [16]:
sorted_damage_df_dict = sort_dataframes_by_index(damage_df_dict)

Dataframe:  grey_matter


,07_default.nii,01_visual_seed.nii,05_limbic.nii,02_somatomotor_seed.nii,03_dorsal_attention.nii,04_ventral_attention.nii,06_frontoparietal.nii,Total Atrophy Voxels
101,-6270.805613,-4621.706587,-2785.906135,-2547.638039,-2233.348331,-1605.735372,-2763.053388,8731.0
102,-6382.296963,-5233.405794,-5464.135222,-2112.714091,-2261.148821,-2600.791809,-3898.895227,12827.0
103,-6511.410148,-3184.230665,-3188.958556,-3057.858744,-1572.827957,-2781.618333,-3434.339218,10463.0
104,-8253.512304,-3775.932704,-4279.926495,-2730.621932,-2950.0213,-3027.468769,-3825.728658,11277.0
105,-5079.024836,-2082.808661,-2442.523424,-2219.865289,-2348.989669,-1926.871739,-3729.452513,7913.0
106,-2171.991575,-2394.039382,-1181.315459,-540.780031,-1488.60791,-797.081924,-1820.903033,4426.0
107,-7057.018514,-6628.774196,-5403.538442,-4475.825124,-3829.888645,-2505.786656,-3740.9724,13976.0
108,-8107.05032,-4703.629885,-3389.400475,-3244.698615,-2471.534292,-1827.228059,-2678.637152,10519.0
109,-6662.306683,-5023.988879,-4740.987264,-2265.663935,-2041.301181,-2479.113512,-3235.212698,9663.0
110,-5984.687399,-3122.84165,-3279.930669,-3785.09862,-2920.439658,-3864.736166,-4290.159793,10973.0


------------- 

Dataframe:  white_matter


,07_default.nii,01_visual_seed.nii,05_limbic.nii,02_somatomotor_seed.nii,03_dorsal_attention.nii,04_ventral_attention.nii,06_frontoparietal.nii,Total Atrophy Voxels
101,-338.814533,-207.506832,-350.510831,-370.559216,-194.979006,-222.541308,-202.642969,1120.0
102,-1665.461832,-845.61242,-983.38329,-1164.971641,-565.280937,-1002.061902,-1133.265143,8175.0
103,-1243.347499,-534.992898,-314.468758,-745.359549,-348.229334,-664.798577,-673.528106,3439.0
104,-788.162714,-567.140507,-390.897839,-500.467811,-334.420579,-435.080464,-551.457013,2147.0
105,-760.086842,-504.407785,-263.43503,-482.673553,-348.600226,-495.162685,-711.020586,1770.0
106,-1223.273812,-831.893366,-538.410954,-358.271343,-418.733141,-380.559003,-680.509347,2343.0
107,-994.79027,-682.535048,-565.295653,-413.534831,-564.389664,-380.022258,-631.275725,3050.0
108,-1811.705724,-920.876664,-746.509263,-2862.964577,-410.554899,-1052.096194,-572.920419,4757.0
109,-582.022142,-794.321683,-376.252447,-573.953527,-464.076117,-462.652188,-209.10823,2654.0
110,-820.166294,-618.271922,-341.262877,-506.189553,-407.280241,-455.32202,-476.107481,2262.0


------------- 

Dataframe:  cerebrospinal_fluid


,07_default.nii,01_visual_seed.nii,05_limbic.nii,02_somatomotor_seed.nii,03_dorsal_attention.nii,04_ventral_attention.nii,06_frontoparietal.nii,Total Atrophy Voxels
101,15310.064023,13137.737754,5308.167206,6530.137526,8517.089994,4291.447046,7349.215393,15272.0
102,5481.951885,6029.077184,2718.037027,1258.052297,2936.173831,1702.040542,4636.005165,7029.0
103,13945.33584,8700.792311,4590.19141,4528.44292,4367.906299,5476.531524,8037.637032,12868.0
104,9539.651087,8345.846932,4920.344383,3040.712396,5039.257616,4016.944564,6271.056899,10426.0
105,2904.552208,1549.510146,930.005487,1724.525596,1552.213374,1669.565872,3330.199847,3149.0
106,2690.561112,3154.049577,468.856877,836.618023,1427.032366,630.12343,2411.333388,3032.0
107,4426.463716,5920.364036,2389.605977,2800.250174,3464.756045,1999.907275,2767.007182,6087.0
108,11916.576205,11953.440405,5247.425554,9444.495133,5772.577113,6165.476253,8286.414612,13124.0
109,17923.662841,9313.825373,6170.661867,9467.661483,8205.380507,9089.650792,13307.331159,21124.0
110,16086.397284,10736.411422,5237.428643,6741.741356,7373.290814,7257.964208,13073.054677,18654.0


------------- 

Dataframe:  composite


,07_default.nii,01_visual_seed.nii,05_limbic.nii,02_somatomotor_seed.nii,03_dorsal_attention.nii,04_ventral_attention.nii,06_frontoparietal.nii,Total Atrophy Voxels
101,21903.170031,17957.673191,8404.868227,9439.419785,10936.447981,6115.085079,10314.911749,20849.0
102,13515.463375,12082.924578,9118.679388,4527.835919,5743.538218,5302.068042,9656.458862,24230.0
103,21652.451729,12374.05669,8076.577222,8310.838732,6284.733216,8894.344195,12129.299072,22327.0
104,18530.664997,12635.981344,9532.447564,6255.09139,8310.157882,7465.481738,10620.218267,19756.0
105,8733.179688,4116.594017,3633.388726,4404.165401,4249.803269,4081.093058,7760.546061,11670.0
106,6082.399537,6377.536797,2188.58329,1735.669396,3331.95168,1807.764356,4907.161461,8922.0
107,12465.439979,13215.160937,8343.393797,7678.161769,7852.485617,4876.8917,7134.532788,20189.0
108,21710.049405,17434.430746,9310.343987,15141.336205,8642.844347,8945.818585,11515.875431,23894.0
109,25161.368954,15101.270296,11257.506771,12292.241319,10700.715261,12022.943504,16740.088351,28436.0
110,22859.824946,14388.740816,8819.016389,11013.832386,10687.020706,11548.361338,17829.459363,26745.0


------------- 



Save the Results

In [17]:
import os
from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti
from tqdm import tqdm

def save_csv_to_bids(dataframes_dict, bids_base_dir, analysis='atrophy_results', ses=None, dry_run=True):
    """
    Saves csv to a BIDS directory structure.
    
    Parameters:
    - dataframes_dict (dict): Dictionary containing dataframes with NIFTI data.
    - bids_base_dir (str): The base directory where the BIDS structure starts.
    - ses (str, optional): Session identifier. If None, defaults to '01'.
    
    Note:
    This function assumes a predefined BIDS directory structure and saves the CSV 
    accordingly.
    """
    
    for key, value in tqdm(dataframes_dict.items()):            
        # Define BIDS Directory Architecture
        sub_no = 'all'
        if ses is None:
            ses_no = '01'
        else:
            ses_no = ses
        
        # Define and Initialize the Save Directory
        out_dir = os.path.join(bids_base_dir, 'neuroimaging_analyses', f'ses-{ses_no}', f'sub-{sub_no}', analysis)
        os.makedirs(out_dir, exist_ok=True)
        
        # Save Image to BIDS Directory
        if dry_run:
            print(out_dir+f'/{key}.csv')
        else:
            value.to_csv(out_dir+f'/{key}.csv')
            print('Saved to: ', out_dir+f'/{key}.csv')

In [19]:
bids_base_dir = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX'
analysis='yeo_roi_atrophy_results'

In [21]:
save_csv_to_bids(sorted_damage_df_dict, bids_base_dir, analysis, ses=None, dry_run=False)

100%|██████████| 4/4 [00:00<00:00, 396.87it/s]

Saved to:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/neuroimaging_analyses/ses-01/sub-all/yeo_roi_atrophy_results/grey_matter.csv
Saved to:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/neuroimaging_analyses/ses-01/sub-all/yeo_roi_atrophy_results/white_matter.csv
Saved to:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/neuroimaging_analyses/ses-01/sub-all/yeo_roi_atrophy_results/cerebrospinal_fluid.csv
Saved to:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/neuroimaging_analyses/ses-01/sub-all/yeo_roi_atrophy_results/composite.csv


All done. Enjoy your analyses. 

--Calvin 